In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

from datetime import datetime
from functools import partial
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torchvision.models import resnet
from tqdm import tqdm
import argparse
import json
import math
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import lightly.models as models
import lightly.loss as loss

os.environ["CUDA_VISIBLE_DEVICES"]="1"

Sun Jun 27 21:28:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN V             Off  | 00000000:3B:00.0 Off |                  N/A |
| 38%   46C    P2    48W / 250W |  11106MiB / 12066MiB |     12%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN V             Off  | 00000000:5E:00.0 Off |                  N/A |
| 28%   

/home/l/liny/anaconda3/envs/mypy37-2/lib/python3.7/site-packages/pytorch_lightning/metrics/__init__.py:44: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  "`pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package"


## Create dataloader, model

In [5]:
parser = argparse.ArgumentParser(description='Train SimSiam on Logo targetlist')

parser.add_argument('-a', '--arch', default='resnet18')

# lr: 0.06 for batch 512 (or 0.03 for batch 256)
parser.add_argument('--lr', '--learning-rate', default=0.06, type=float, metavar='LR', help='initial learning rate', dest='lr')
parser.add_argument('--epochs', default=800, type=int, metavar='N', help='number of total epochs to run')
parser.add_argument('--schedule', default=[300, 600], nargs='*', type=int, help='learning rate schedule (when to drop lr by 10x); does not take effect if --cos is on')
parser.add_argument('--cos', action='store_true', help='use cosine lr schedule')

parser.add_argument('--batch-size', default=256, type=int, metavar='N', help='mini-batch size')
parser.add_argument('--wd', default=5e-4, type=float, metavar='W', help='weight decay')

parser.add_argument('--bn-splits', default=8, type=int, help='simulate multi-gpu behavior of BatchNorm in one gpu; 1 is SyncBatchNorm in multi-gpu')

parser.add_argument('--symmetric', action='store_true', help='use a symmetric loss function that backprops to both crops')

# knn monitor
parser.add_argument('--knn-k', default=200, type=int, help='k in kNN monitor')
parser.add_argument('--knn-t', default=0.1, type=float, help='softmax temperature in kNN monitor; could be different with moco-t')

# utils
parser.add_argument('--resume', default='', 
                    type=str, metavar='PATH', help='path to latest checkpoint (default: none)')
parser.add_argument('--results-dir', default='', type=str, metavar='PATH', help='path to cache (default: none)')

'''
args = parser.parse_args()  # running in command line
'''
args = parser.parse_args('')  # running in ipynb

# set command line arguments here when running in ipynb
args.epochs = 800
args.cos = True
args.schedule = []  # cos in use
args.symmetric = False
if args.results_dir == '':
    args.results_dir = '../output/Targetlist-' + datetime.now().strftime("%Y-%m-%d-%H-%M-%S-moco")

print(args)

Namespace(arch='resnet18', batch_size=256, bn_splits=8, cos=True, epochs=800, knn_k=200, knn_t=0.1, lr=0.06, results_dir='../output/Targetlist-2021-06-27-21-29-03-moco', resume='', schedule=[], symmetric=False, wd=0.0005)


In [6]:
import pickle
from PIL import Image, ImageOps
from typing import List, Union, Callable

class GetLoader(torch.utils.data.Dataset):
    '''Define customized dataset
    Args:
        data_root:
            Path to directory holding the images to load.
        data_list:
            Path to txt file which map images to labels.
        label_dict:
            Dict which converts label in plain text to label in int.
        transform:
            Transformations to apply.
        grayscale:
            Grayscale model/RGB model, default is RGB
    
    '''
    def __init__(self, data_root, data_list, label_dict, transform=None, grayscale=False):
        
        self.transform = transform
        self.data_root = data_root
        self.grayscale = grayscale
        data_list = [x.strip('\n') for x in open(data_list).readlines()]

        with open(label_dict, 'rb') as handle:
            self.label_dict = pickle.load(handle)

        self.classes = list(self.label_dict.keys())

        self.n_data = len(data_list)

        self.img_paths = []
        self.labels = []
        self.targets = []

        for data in data_list:
            image_path = data
            label = image_path.split('/')[0]
            self.img_paths.append(image_path)
            self.labels.append(label)
            self.targets.append(self.label_dict[label])

    def __getitem__(self, index):

        img_path, label= self.img_paths[index], self.labels[index]
        img_path_full = os.path.join(self.data_root, img_path)
        if self.grayscale:
            img = Image.open(img_path_full).convert("RGBA").convert('L').convert('RGB')
        else:
            img = Image.open(img_path_full).convert("RGBA").convert('RGB')

        img = ImageOps.expand(img, (
            (max(img.size) - img.size[0]) // 2, (max(img.size) - img.size[1]) // 2,
            (max(img.size) - img.size[0]) // 2, (max(img.size) - img.size[1]) // 2), fill=(255, 255, 255))

        label = self.label_dict[label]
        if self.transform is not None:
            img = self.transform(img)

        return img, label

    def __len__(self):
        return self.n_data


class GetLoaderPair(GetLoader):
    """CIFAR10 Dataset.
    """
    def __getitem__(self, index):
        
        img_path, label = self.img_paths[index], self.labels[index]
        img_path_full = os.path.join(self.data_root, img_path)
        if self.grayscale:
            img = Image.open(img_path_full).convert("RGBA").convert('L').convert('RGB')
        else:
            img = Image.open(img_path_full).convert("RGBA").convert('RGB')

        img = ImageOps.expand(img, (
            (max(img.size) - img.size[0]) // 2, (max(img.size) - img.size[1]) // 2,
            (max(img.size) - img.size[0]) // 2, (max(img.size) - img.size[1]) // 2), fill=(255, 255, 255))

        if self.transform is not None:
            im_1 = self.transform(img)
            im_2 = self.transform(img)

        return im_1, im_2

    
train_transform = transforms.Compose([
    transforms.CenterCrop(128),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

test_transform = transforms.Compose([
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

In [7]:
# data prepare
train_data = GetLoaderPair(data_root='../datasets/targetlist/train/', 
                           data_list='/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/train_targets.txt', 
                           label_dict='/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/target_dict.json',
                           transform=train_transform)

train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=True, 
                          num_workers=16, pin_memory=True, drop_last=True)

In [8]:
memory_data = GetLoader(data_root='../datasets/targetlist/train/', 
                        data_list='/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/train_targets.txt', 
                        label_dict='/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/target_dict.json',
                        transform=test_transform)
memory_loader = DataLoader(memory_data, batch_size=args.batch_size, 
                           shuffle=False, num_workers=16, pin_memory=True)

In [9]:
test_data = GetLoader(data_root='../datasets/targetlist/test/', 
                      data_list='/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/test_targets.txt', 
                      label_dict='/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/target_dict.json',
                      transform=test_transform)

test_loader = DataLoader(test_data, batch_size=args.batch_size, 
                         shuffle=False, num_workers=16, pin_memory=True)

## Model

In [10]:
# use a resnet50 backbone
resnet = torchvision.models.resnet.resnet18()
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])

# build the simsiam model
model = models.SimSiam(resnet, num_ftrs=512)
model = model.cuda()

# use a criterion for self-supervised learning
criterion = loss.SymNegCosineSimilarityLoss()

In [11]:
# train for one epoch
def train(net, data_loader, train_optimizer, epoch, args):
    net.train()
    adjust_learning_rate(optimizer, epoch, args)

    total_loss, total_num, train_bar = 0.0, 0, tqdm(data_loader)
    for im_1, im_2 in train_bar:
        im_1, im_2 = im_1.cuda(non_blocking=True), im_2.cuda(non_blocking=True)

        y0, y1 = net(im_1, im_2)
        # backpropagation
        loss = criterion(y0, y1)
        
        train_optimizer.zero_grad()
        loss.backward()
        train_optimizer.step()

        total_num += data_loader.batch_size
        total_loss += loss.item() * data_loader.batch_size
        train_bar.set_description('Train Epoch: [{}/{}], lr: {:.6f}, Loss: {:.4f}'.format(epoch, args.epochs, optimizer.param_groups[0]['lr'], total_loss / total_num))

    return total_loss / total_num

# lr scheduler for training
def adjust_learning_rate(optimizer, epoch, args):
    """Decay the learning rate based on schedule"""
    lr = args.lr
    if args.cos:  # cosine lr schedule
        lr *= 0.5 * (1. + math.cos(math.pi * epoch / args.epochs))
    else:  # stepwise lr schedule
        for milestone in args.schedule:
            lr *= 0.1 if epoch >= milestone else 1.
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [12]:
# test using a knn monitor
def test(net, memory_data_loader, test_data_loader, epoch, args):
    net.eval()
    classes = len(memory_data_loader.dataset.classes)
    total_top1, total_top5, total_num, feature_bank = 0.0, 0.0, 0, []
    with torch.no_grad():
        # generate feature bank
        for data, target in tqdm(memory_data_loader, desc='Feature extracting'):
            feature = net(data.cuda(non_blocking=True)).squeeze()
            feature = F.normalize(feature, dim=1)
            feature_bank.append(feature)
        # [D, N]
        feature_bank = torch.cat(feature_bank, dim=0).t().contiguous()
        # [N]
        feature_labels = torch.tensor(memory_data_loader.dataset.targets, device=feature_bank.device)
        
        # loop test data to predict the label by weighted knn search
        test_bar = tqdm(test_data_loader)
        for data, target in test_bar:
            data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)
            feature = net(data).squeeze()
            feature = F.normalize(feature, dim=1)
            
            pred_labels = knn_predict(feature, feature_bank, feature_labels, classes, args.knn_k, args.knn_t)

            total_num += data.size(0)
            total_top1 += (pred_labels[:, 0] == target).float().sum().item()
            test_bar.set_description('Test Epoch: [{}/{}] Acc@1:{:.2f}%'.format(epoch, args.epochs, total_top1 / total_num * 100))

    return total_top1 / total_num * 100

# knn monitor as in InstDisc https://arxiv.org/abs/1805.01978
# implementation follows http://github.com/zhirongw/lemniscate.pytorch and https://github.com/leftthomas/SimCLR
def knn_predict(feature, feature_bank, feature_labels, classes, knn_k, knn_t):
    # compute cos similarity between each feature vector and feature bank ---> [B, N]
    sim_matrix = torch.mm(feature, feature_bank)
    # [B, K]
    sim_weight, sim_indices = sim_matrix.topk(k=knn_k, dim=-1)
    # [B, K]
    sim_labels = torch.gather(feature_labels.expand(feature.size(0), -1), dim=-1, index=sim_indices)
    sim_weight = (sim_weight / knn_t).exp()

    # counts for each class
    one_hot_label = torch.zeros(feature.size(0) * knn_k, classes, device=sim_labels.device)
    # [B*K, C]
    one_hot_label = one_hot_label.scatter(dim=-1, index=sim_labels.view(-1, 1), value=1.0)
    # weighted score ---> [B, C]
    pred_scores = torch.sum(one_hot_label.view(feature.size(0), -1, classes) * sim_weight.unsqueeze(dim=-1), dim=1)

    pred_labels = pred_scores.argsort(dim=-1, descending=True)
    return pred_labels

In [13]:
# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.wd, momentum=0.9)

# load model if resume
epoch_start = 1
if args.resume is not '':
    checkpoint = torch.load(args.resume)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch_start = checkpoint['epoch'] + 1
    print('Loaded from: {}'.format(args.resume))

In [14]:
# logging
results = {'train_loss': [], 'test_acc@1': []}
if not os.path.exists(args.results_dir):
    os.mkdir(args.results_dir)
    
# dump args
with open(args.results_dir + '/args.json', 'w') as fid:
    json.dump(args.__dict__, fid, indent=2)


In [15]:
# training loop
for epoch in range(epoch_start, args.epochs + 1):
    test_acc_1 = test(model.backbone, memory_loader, test_loader, epoch, args)
    train_loss = train(model, train_loader, optimizer, epoch, args)
    results['train_loss'].append(train_loss)
    test_acc_1 = test(model.backbone, memory_loader, test_loader, epoch, args)
    results['test_acc@1'].append(test_acc_1)
    # save statistics
    data_frame = pd.DataFrame(data=results, index=range(epoch_start, epoch + 1))
    data_frame.to_csv(args.results_dir + '/log.csv', index_label='epoch')
    # save model
    torch.save({'epoch': epoch, 'state_dict': model.state_dict(), 'optimizer' : optimizer.state_dict(),}, args.results_dir + '/model_last.pth')

Test Epoch: [1/800] Acc@1:5.57%: 100%|██████████| 3/3 [00:08<00:00,  2.71s/it]
Train Epoch: [1/800], lr: 0.060000, Loss: -0.0149: 100%|██████████| 9/9 [00:10<00:00,  1.15s/it]
Test Epoch: [2/800] Acc@1:6.93%: 100%|██████████| 3/3 [00:05<00:00,  1.88s/it]
Train Epoch: [2/800], lr: 0.059999, Loss: -0.0794: 100%|██████████| 9/9 [00:09<00:00,  1.09s/it]
Test Epoch: [3/800] Acc@1:7.26%: 100%|██████████| 3/3 [00:06<00:00,  2.06s/it]
Train Epoch: [3/800], lr: 0.059998, Loss: -0.2512: 100%|██████████| 9/9 [00:10<00:00,  1.13s/it]
Test Epoch: [4/800] Acc@1:5.91%: 100%|██████████| 3/3 [00:06<00:00,  2.12s/it]
Train Epoch: [4/800], lr: 0.059996, Loss: -0.5159: 100%|██████████| 9/9 [00:11<00:00,  1.30s/it]
Test Epoch: [5/800] Acc@1:6.42%: 100%|██████████| 3/3 [00:05<00:00,  1.94s/it]
Train Epoch: [5/800], lr: 0.059994, Loss: -0.6994: 100%|██████████| 9/9 [00:10<00:00,  1.21s/it]
Test Epoch: [6/800] Acc@1:5.24%: 100%|██████████| 3/3 [00:06<00:00,  2.03s/it]
Train Epoch: [6/800], lr: 0.059992, Loss:

Test Epoch: [22/800] Acc@1:4.22%: 100%|██████████| 3/3 [00:06<00:00,  2.33s/it]
Train Epoch: [22/800], lr: 0.059888, Loss: -0.9047: 100%|██████████| 9/9 [00:09<00:00,  1.04s/it]
Test Epoch: [23/800] Acc@1:4.22%: 100%|██████████| 3/3 [00:05<00:00,  2.00s/it]
Train Epoch: [23/800], lr: 0.059878, Loss: -0.9013: 100%|██████████| 9/9 [00:10<00:00,  1.18s/it]
Test Epoch: [24/800] Acc@1:4.56%: 100%|██████████| 3/3 [00:05<00:00,  1.81s/it]
Train Epoch: [24/800], lr: 0.059867, Loss: -0.8958: 100%|██████████| 9/9 [00:12<00:00,  1.40s/it]
Test Epoch: [25/800] Acc@1:5.07%: 100%|██████████| 3/3 [00:07<00:00,  2.55s/it]
Train Epoch: [25/800], lr: 0.059856, Loss: -0.9014: 100%|██████████| 9/9 [00:10<00:00,  1.16s/it]
Test Epoch: [26/800] Acc@1:5.41%: 100%|██████████| 3/3 [00:06<00:00,  2.01s/it]
Train Epoch: [26/800], lr: 0.059844, Loss: -0.8974: 100%|██████████| 9/9 [00:12<00:00,  1.41s/it]
Test Epoch: [27/800] Acc@1:4.90%: 100%|██████████| 3/3 [00:06<00:00,  2.15s/it]
Train Epoch: [27/800], lr: 0.0

Train Epoch: [42/800], lr: 0.059593, Loss: -0.9166: 100%|██████████| 9/9 [00:08<00:00,  1.08it/s]
Test Epoch: [43/800] Acc@1:4.56%: 100%|██████████| 3/3 [00:06<00:00,  2.12s/it]
Train Epoch: [43/800], lr: 0.059573, Loss: -0.9142: 100%|██████████| 9/9 [00:11<00:00,  1.32s/it]
Test Epoch: [44/800] Acc@1:4.90%: 100%|██████████| 3/3 [00:07<00:00,  2.58s/it]
Train Epoch: [44/800], lr: 0.059553, Loss: -0.9124: 100%|██████████| 9/9 [00:11<00:00,  1.24s/it]
Test Epoch: [45/800] Acc@1:4.90%: 100%|██████████| 3/3 [00:05<00:00,  1.75s/it]
Train Epoch: [45/800], lr: 0.059533, Loss: -0.9148: 100%|██████████| 9/9 [00:12<00:00,  1.36s/it]
Test Epoch: [46/800] Acc@1:5.07%: 100%|██████████| 3/3 [00:05<00:00,  1.83s/it]
Train Epoch: [46/800], lr: 0.059512, Loss: -0.9182: 100%|██████████| 9/9 [00:09<00:00,  1.08s/it]
Test Epoch: [47/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:05<00:00,  1.77s/it]
Train Epoch: [47/800], lr: 0.059490, Loss: -0.9173: 100%|██████████| 9/9 [00:09<00:00,  1.10s/it]
Test Epoch: 

Test Epoch: [63/800] Acc@1:6.08%: 100%|██████████| 3/3 [00:07<00:00,  2.59s/it]
Train Epoch: [63/800], lr: 0.059087, Loss: -0.9374: 100%|██████████| 9/9 [00:11<00:00,  1.30s/it]
Test Epoch: [64/800] Acc@1:4.90%: 100%|██████████| 3/3 [00:05<00:00,  1.90s/it]
Train Epoch: [64/800], lr: 0.059057, Loss: -0.9398: 100%|██████████| 9/9 [00:09<00:00,  1.09s/it]
Test Epoch: [65/800] Acc@1:5.07%: 100%|██████████| 3/3 [00:05<00:00,  1.95s/it]
Train Epoch: [65/800], lr: 0.059028, Loss: -0.9401: 100%|██████████| 9/9 [00:10<00:00,  1.17s/it]
Test Epoch: [66/800] Acc@1:5.07%: 100%|██████████| 3/3 [00:05<00:00,  1.78s/it]
Train Epoch: [66/800], lr: 0.058998, Loss: -0.9424: 100%|██████████| 9/9 [00:09<00:00,  1.08s/it]
Test Epoch: [67/800] Acc@1:5.57%: 100%|██████████| 3/3 [00:05<00:00,  1.86s/it]
Train Epoch: [67/800], lr: 0.058968, Loss: -0.9463: 100%|██████████| 9/9 [00:11<00:00,  1.28s/it]
Test Epoch: [68/800] Acc@1:4.90%: 100%|██████████| 3/3 [00:07<00:00,  2.50s/it]
Train Epoch: [68/800], lr: 0.0

Test Epoch: [84/800] Acc@1:5.07%: 100%|██████████| 3/3 [00:08<00:00,  2.92s/it]
Train Epoch: [84/800], lr: 0.058383, Loss: -0.9423: 100%|██████████| 9/9 [00:10<00:00,  1.12s/it]
Test Epoch: [85/800] Acc@1:5.91%: 100%|██████████| 3/3 [00:05<00:00,  2.00s/it]
Train Epoch: [85/800], lr: 0.058344, Loss: -0.9434: 100%|██████████| 9/9 [00:09<00:00,  1.05s/it]
Test Epoch: [86/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:08<00:00,  2.73s/it]
Train Epoch: [86/800], lr: 0.058305, Loss: -0.9393: 100%|██████████| 9/9 [00:10<00:00,  1.14s/it]
Test Epoch: [87/800] Acc@1:4.73%: 100%|██████████| 3/3 [00:05<00:00,  1.92s/it]
Train Epoch: [87/800], lr: 0.058266, Loss: -0.9448: 100%|██████████| 9/9 [00:10<00:00,  1.21s/it]
Test Epoch: [88/800] Acc@1:4.73%: 100%|██████████| 3/3 [00:07<00:00,  2.51s/it]
Train Epoch: [88/800], lr: 0.058226, Loss: -0.9430: 100%|██████████| 9/9 [00:09<00:00,  1.03s/it]
Test Epoch: [89/800] Acc@1:5.41%: 100%|██████████| 3/3 [00:05<00:00,  1.93s/it]
Train Epoch: [89/800], lr: 0.0

...

Test Epoch: [707/800] Acc@1:4.22%: 100%|██████████| 3/3 [00:08<00:00,  2.96s/it]
Train Epoch: [707/800], lr: 0.001979, Loss: -0.9898: 100%|██████████| 9/9 [00:12<00:00,  1.37s/it]
Test Epoch: [708/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:07<00:00,  2.57s/it]
Train Epoch: [708/800], lr: 0.001937, Loss: -0.9897: 100%|██████████| 9/9 [00:09<00:00,  1.11s/it]
Test Epoch: [709/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:06<00:00,  2.02s/it]
Train Epoch: [709/800], lr: 0.001895, Loss: -0.9899: 100%|██████████| 9/9 [00:09<00:00,  1.04s/it]
Test Epoch: [710/800] Acc@1:4.22%: 100%|██████████| 3/3 [00:05<00:00,  1.87s/it]
Train Epoch: [710/800], lr: 0.001854, Loss: -0.9895: 100%|██████████| 9/9 [00:09<00:00,  1.08s/it]
Test Epoch: [711/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:05<00:00,  1.99s/it]
Train Epoch: [711/800], lr: 0.001814, Loss: -0.9886: 100%|██████████| 9/9 [00:10<00:00,  1.16s/it]
Test Epoch: [712/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:05<00:00,  1.83s/it]
Train Epoch: [712/8

Test Epoch: [728/800] Acc@1:4.56%: 100%|██████████| 3/3 [00:05<00:00,  1.86s/it]
Train Epoch: [728/800], lr: 0.001191, Loss: -0.9900: 100%|██████████| 9/9 [00:08<00:00,  1.00it/s]
Test Epoch: [729/800] Acc@1:4.56%: 100%|██████████| 3/3 [00:05<00:00,  1.68s/it]
Train Epoch: [729/800], lr: 0.001159, Loss: -0.9903: 100%|██████████| 9/9 [00:12<00:00,  1.44s/it]
Test Epoch: [730/800] Acc@1:4.90%: 100%|██████████| 3/3 [00:05<00:00,  1.84s/it]
Train Epoch: [730/800], lr: 0.001126, Loss: -0.9908: 100%|██████████| 9/9 [00:09<00:00,  1.08s/it]
Test Epoch: [731/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:05<00:00,  1.93s/it]
Train Epoch: [731/800], lr: 0.001095, Loss: -0.9909: 100%|██████████| 9/9 [00:12<00:00,  1.37s/it]
Test Epoch: [732/800] Acc@1:4.90%: 100%|██████████| 3/3 [00:12<00:00,  4.32s/it]
Train Epoch: [732/800], lr: 0.001063, Loss: -0.9903: 100%|██████████| 9/9 [00:09<00:00,  1.09s/it]
Test Epoch: [733/800] Acc@1:4.73%: 100%|██████████| 3/3 [00:05<00:00,  1.86s/it]
Train Epoch: [733/8

Train Epoch: [748/800], lr: 0.000623, Loss: -0.9898: 100%|██████████| 9/9 [00:09<00:00,  1.09s/it]
Test Epoch: [749/800] Acc@1:4.56%: 100%|██████████| 3/3 [00:05<00:00,  1.73s/it]
Train Epoch: [749/800], lr: 0.000600, Loss: -0.9893: 100%|██████████| 9/9 [00:12<00:00,  1.40s/it]
Test Epoch: [750/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:08<00:00,  2.98s/it]
Train Epoch: [750/800], lr: 0.000576, Loss: -0.9888: 100%|██████████| 9/9 [00:08<00:00,  1.00it/s]
Test Epoch: [751/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:05<00:00,  1.89s/it]
Train Epoch: [751/800], lr: 0.000554, Loss: -0.9891: 100%|██████████| 9/9 [00:13<00:00,  1.52s/it]
Test Epoch: [752/800] Acc@1:4.22%: 100%|██████████| 3/3 [00:08<00:00,  2.89s/it]
Train Epoch: [752/800], lr: 0.000531, Loss: -0.9891: 100%|██████████| 9/9 [00:11<00:00,  1.27s/it]
Test Epoch: [753/800] Acc@1:4.22%: 100%|██████████| 3/3 [00:05<00:00,  2.00s/it]
Train Epoch: [753/800], lr: 0.000510, Loss: -0.9886: 100%|██████████| 9/9 [00:08<00:00,  1.06it/s]
T

Test Epoch: [769/800] Acc@1:4.22%: 100%|██████████| 3/3 [00:07<00:00,  2.45s/it]
Train Epoch: [769/800], lr: 0.000222, Loss: -0.9868: 100%|██████████| 9/9 [00:09<00:00,  1.07s/it]
Test Epoch: [770/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:05<00:00,  1.81s/it]
Train Epoch: [770/800], lr: 0.000208, Loss: -0.9861: 100%|██████████| 9/9 [00:14<00:00,  1.58s/it]
Test Epoch: [771/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:07<00:00,  2.55s/it]
Train Epoch: [771/800], lr: 0.000194, Loss: -0.9871: 100%|██████████| 9/9 [00:10<00:00,  1.14s/it]
Test Epoch: [772/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:05<00:00,  1.95s/it]
Train Epoch: [772/800], lr: 0.000181, Loss: -0.9867: 100%|██████████| 9/9 [00:10<00:00,  1.13s/it]
Test Epoch: [773/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:04<00:00,  1.41s/it]
Train Epoch: [773/800], lr: 0.000168, Loss: -0.9858: 100%|██████████| 9/9 [00:11<00:00,  1.24s/it]
Test Epoch: [774/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:06<00:00,  2.02s/it]
Train Epoch: [774/8

Test Epoch: [790/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:05<00:00,  1.95s/it]
Train Epoch: [790/800], lr: 0.000023, Loss: -0.9865: 100%|██████████| 9/9 [00:10<00:00,  1.12s/it]
Test Epoch: [791/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:05<00:00,  1.83s/it]
Train Epoch: [791/800], lr: 0.000019, Loss: -0.9856: 100%|██████████| 9/9 [00:10<00:00,  1.18s/it]
Test Epoch: [792/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:06<00:00,  2.11s/it]
Train Epoch: [792/800], lr: 0.000015, Loss: -0.9863: 100%|██████████| 9/9 [00:11<00:00,  1.25s/it]
Test Epoch: [793/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:05<00:00,  1.81s/it]
Train Epoch: [793/800], lr: 0.000011, Loss: -0.9857: 100%|██████████| 9/9 [00:09<00:00,  1.11s/it]
Test Epoch: [794/800] Acc@1:4.39%: 100%|██████████| 3/3 [00:05<00:00,  1.88s/it]
Train Epoch: [794/800], lr: 0.000008, Loss: -0.9853: 100%|██████████| 9/9 [00:12<00:00,  1.39s/it]
Test Epoch: [795/800] Acc@1:4.56%: 100%|██████████| 3/3 [00:08<00:00,  2.73s/it]
Train Epoch: [795/8